# Neighborhood Computations

In [1]:
import numpy as np
import igl
import meshplot

In [2]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off") # reading the file in as an off 
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj") # reading the file in as an obj 
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

In [3]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [4]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [5]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [6]:
print(bunny_v.shape[0])

3485


## Simple Neighborhood Calculations - Notes
- Mesh can be considered as an undirected graph
- To pick faces that are neighboring to a vertex, you can go through all the faces, and see which one includes the vertex we're looking at (one option)
- In this assignemnt, we will use vertex adjecencies

Required output of this section:
- A text dump of the content of the two data structures for the provided mesh bunny_small.off.

### Vertex-to-Face Relations
Given V and F, generate an adjacency list which contains, for each vertex, a list of faces adjacent to it. The ordering of the faces incident on a vertex does not matter. Your program should print out the vertex-to-face relations.

Relevant `igl` functions: `igl.vertex_triangle_adjacency`

In [7]:
"""
We want a list of faces per each vertex.
VF (list w/ dimension 3*#F) 
- List of faces indice on each vertex, so that VF(NI(i)+j) =
f, means that face f is the jth face (in no particular order) incident
on vertex i.
NI (list w/ dim #V+1)
- List of the cumulative sum of vertex-triangle degrees with a
preceeding zero. “How many faces” have been seen before visiting this
vertex and its incident faces.
"""
V = bunny_v
F = bunny_f
VF, NI = igl.vertex_triangle_adjacency(F, V.shape[0]); 
# had to use shape because F.maxcoeff had error: 'numpy.ndarray' object has no attribute 'maxCoeff'

In [8]:
print(VF) # flattened array of face indeces adjacent to each vertex 
print(NI) # array with cumulative sum of degrees per vertex

nv = V.shape[0] # number of vertices
#NI[1] = degree of vertex 1, NI[2]-NI[1] = degree of vertex 2
# print(NI[1])
# print(NI[2])
# print(NI[3])
adjacency_list = {}
for i in range(1, nv+1): # for each vertex 
    degree = NI[i]-NI[i-1] # degree of vertex, i.
    face_list = []
    for j in range(degree): # get the faces 
        face_list.append(int(VF[NI[i-1]+j])) # NI[i-1] because that's the index of the vertex?
    adjacency_list[i] = face_list

#print(adjacency_list)

[ 849  850  912 ... 1500 5112 5264]
[    0     5     9 ... 20884 20891 20898]


### Vertex-to-Vertex Relations
Given V and F, generate an adjacency list which contains, for each vertex, a list of vertices connected with it. Two vertices are connected if there exists an edge between them, i.e., if the two vertices appear in the same row of F. The ordering of the vertices in each list does not matter. Your program should print out the vertex-to-vertex relations.

Relevant igl functions: igl.adjacency_list.

In [9]:
V = bunny_v
F = bunny_f
a = igl.adjacency_list(F)

In [14]:
#print(a)

### Visualizing the Neighborhood Relations

Check your results by comparing them to raw data inside F.

A text dump of the content of the two data structures for the provided mesh bunny_small.off.

In order to check, I will consider the fact that the number of adjacent vertices to a vertex = number of incident edges to that vertex. If I calculated these correctly, and they're equal.  

In [10]:
lengths_a = [];
for i in a: 
    lengths_a.append(len(i))

In [11]:
lengths_adj_list = [];
for i in adjacency_list: 
    lengths_adj_list.append(len(adjacency_list[i]))

In [12]:
print(lengths_a == lengths_adj_list)

True


## Shading

For this task, you will experiment with the different ways of shading discrete surfaces already implemented in `igl`. Display the mesh with the appropriate shading.

Use `meshplot.plot(v,f, n=n, shading={"flat": False})` to set the shading in the viewer to use the normals `n` you computed.

You can visualise normals `n` by using meshplot as following

```
p = meshplot.plot(v, f, n=n, shading={"flat": False})
p.add_lines(v, v + 0.01*n)
```

The scaling `0.01` is used to scale the lines to a reasonable size; that is, for the cube `0.5` is ok, while for the sphere `0.01` is better.

Note: `meshplot` supports only per-vertex normals; thus, to visualize the different shading you will need to "explode" the mesh. That is, separate all faces and duplicate vertices. For instance, if the mesh has faces `f=[[0, 1, 2], [1, 3, 2]]` (with vertices `1` and `2` shared among the two faces), `exploded_f=[[0, 1, 2], [3, 4, 5]]` with the vertices `3` and `5` being a copy of vertices `1` and `2`. Note that `igl` will give you per-vertex, per-face, and per-vertex-per-face quantities, so you will need to compute and store an index mapping from the input mesh to the exploded one.

#### Meshplot requires per vertex normals, so we need to "explode" the mesh

Required output of this section:
- Screenshots of the provided meshes shaded with flat, per-vertex, and per-corner normals.


In [18]:
print(bunny_f)

[[2784 2497 2027]
 [1077  225 1060]
 [ 425  450  381]
 ...
 [3086 3203 3162]
 [3086 3162 3151]
 [3086 3151 3085]]


In [31]:
bunny_vertex_mapping = {}
exploded_bunny = np.copy(bunny_f)
new_number = max(bunny_f.flatten())+1
for i in range(bunny_f.shape[0]):
    for j in range(3):
        vertex = bunny_f[i,j]
        if vertex not in bunny_vertex_mapping:
            bunny_vertex_mapping[vertex] = [vertex]
        else:
            exploded_bunny[i,j]=new_number
            bunny_vertex_mapping[vertex].append(new_number)
            new_number = new_number +1

In [156]:
def explode(mesh_f, mesh_v):
    mesh_vertex_mapping = {}
    exploded_mesh = np.copy(mesh_f)
    exploded_mesh_v = np.copy(mesh_v)
    vertices_to_copy = []
    new_number = max(mesh_f.flatten())+1
    for i in range(mesh_f.shape[0]):
        for j in range(3):
            vertex = mesh_f[i,j]
            if vertex not in mesh_vertex_mapping:
                mesh_vertex_mapping[vertex] = [vertex]
            else:
                exploded_mesh[i,j]=new_number
                vertices_to_copy.append(vertex)
                mesh_vertex_mapping[vertex].append(new_number)
                new_number = new_number +1
    # print(vertices_to_copy)
    for i in vertices_to_copy:
        exploded_mesh_v = np.concatenate((exploded_mesh_v, exploded_mesh_v[i].reshape(1, -1)))
    return exploded_mesh, exploded_mesh_v, mesh_vertex_mapping

In [157]:
 exploded_bf, exploded_bv, exploded_mapb = explode(bunny_f, bunny_v)
 exploded_cf, exploded_cv, exploded_mapc = explode(cube_f, cube_v)

### Flat Shading
The simplest shading technique is flat shading, where each polygon of an object is colored based on the angle between the polygon's surface normal and the direction of the light source, their respective colors, and the intensity of the light source. With flat shading, all vertices of a polygon are colored identically. Your program should compute the appropriate shading normals and shade the input mesh with flat shading.

Relevant igl functions: `igl.per_face_normals`.

In [172]:
"""
per_face_normals(v: array, f: array, z: array)

V #V by 3 eigen Matrix of mesh vertex 3D positions
F #F by 3 eigen Matrix of face (triangle) indices
Z 3 vector normal given to faces with degenerate normal.

Returns	N #F by 3 eigen Matrix of mesh face (triangle) 3D normals
"""
# z = Vector3d(1,1,1).normalized() # Give degenerate faces (1/3,1/3,1/3)^0.5
def flatShade_old(v,f):
    z = np.array([1/3,1/3,1/3])
    v = v
    f = f
    n = igl.per_face_normals(v, f, z)
    print(n)
    print(v.shape)
    print(f.shape)
    p = meshplot.plot(v, f, n=n, shading={"flat": False})
    p.add_lines(v, v + 0.01*n)

"""
flatShade shades the model based on the normals by 
    1. computing the face normals
    2. assign the vertices the normals of the face 
"""
def flatShade(v,f):
    z = np.array([1/3,1/3,1/3])
    v = v
    f = f
    n = igl.per_face_normals(v, f, z)
    new_n = np.copy(v) 
    # loop through all the normals -> get face index (corresponding to normal) -> find vertices for face -> save to new normals array
    for idx, normal in enumerate(n):
        for vert in f[idx]:
            new_n[vert] = normal
    print(n)
    print(new_n.shape)
    print(v.shape)
    print(f.shape)
    p = meshplot.plot(v, f, n=new_n, shading={"flat": False})
    # p.add_lines(v, v + 0.01*new_n)
    
flatShade(exploded_bv, exploded_bf)
flatShade(exploded_cv, exploded_cf)

[[-0.01974875  0.09941629 -0.99484993]
 [-0.98445225  0.06614112 -0.16272408]
 [-0.22538367  0.97105059 -0.07913883]
 ...
 [ 0.14012873 -0.98286055 -0.11978763]
 [ 0.31463126 -0.94851803  0.03634166]
 [ 0.13151583 -0.97764113 -0.16407743]]
(20898, 3)
(20898, 3)
(6966, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

[[ 0.  0.  1.]
 [-0.  0.  1.]
 [-0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0. -1.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]
 [ 1. -0.  0.]
 [ 1.  0.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
(36, 3)
(36, 3)
(12, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-vertex Shading
Flat shading may produce visual artifacts, due to the color discontinuity between neighboring faces. Specular highlights may be rendered poorly with flat shading. When per-vertex shading is used, per-vertex normals are computed for each vertex by averaging the normals of the surrounding faces. Your program should compute the appropriate shading normals and shade the input mesh with per-vertex shading.

Relevant igl functions: igl.per_vertex_normals.

In [179]:
"""
per_vertex_normals(v: array, f: array, z: array)

V #V by 3 eigen Matrix of mesh vertex 3D positions
F #F by 3 eigen Matrix of face (triangle) indices
weighting type 
    -igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM uniform influence
    -igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA area weighted
    -igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_ANGLE angle weighted
    
Returns	N, #V by 3 eigen Matrix of mesh face (triangle) 3D normals
"""
# z = Vector3d(1,1,1).normalized() # Give degenerate faces (1/3,1/3,1/3)^0.5
def gouraudShade(v,f):
    z = np.array([1/3,1/3,1/3])
    v = v
    f = f
    n = igl.per_vertex_normals(v, f, igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)
    print(n)
    print(v.shape)
    print(f.shape)
    p = meshplot.plot(v, f, n=n, shading={"flat": False, "wireframe": True})
    p.add_lines(v, v + 0.005*n)

gouraudShade(bunny_v, bunny_f)

[[-0.3244438   0.63479412  0.70126504]
 [ 0.69665946  0.04959323  0.71568576]
 [-0.31520596  0.88800006  0.33481502]
 ...
 [ 0.43044974 -0.89412873  0.12347808]
 [-0.43673556 -0.78064839 -0.44704602]
 [ 0.03755881 -0.70155862 -0.71162128]]
(3485, 3)
(6966, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### Per-corner Shading
On models with sharp feature lines, averaging the per-face normals on the feature, as done for per-vertex shading, may result in blurred rendering. It is possible to avoid this limitation and to render crisp sharp features by using per-corner normals. In this case, a different normal is assigned to each face corner; this implies that every vertex will get a (possibly different) normal for every adjacent face. A threshold parameter is used to decide when an edge belongs to a sharp feature. The threshold is applied to the angle between the two corner normals: if it is less than the threshold value, the normals must be averaged, otherwise they are kept untouched. Your program should compute the appropriate shading normals (with a threshold of your choice) and shade the input mesh with per-vertex shading.

Relevant igl functions: igl.per_corner_normals.

Compare the result must be compared with the one obtained with flat and per-vertex shading. Experiment with the threshold value.

Required output of this section:

Screenshots of the provided meshes shaded with flat, per-vertex, and per-corner normals.

## Connected Components
Using neighborhood connectivity, it is possible to partition a mesh into connected components, where each mesh face belongs only to a single component. Fill in the appropriate source code sections to display the mesh with each face colored to indicate the component it belongs to (coloring each component distinctly).

Relevant igl functions: igl.face_components

Call meshplot.plot(v, f, c=c), where c is the computed labels to display colors to the per-face colors you computed.

Required output of this section:
- Screenshots of the provided meshes with each connected component colored differently.
- The number of connected components and the size of each component (measured in number of faces) for all the provided models.

## A simple subdivision scheme
![](img/sqrt.png?raw=true)

√3 Subdivision. From left to right: original mesh, added
vertices at the midpoints of the faces (step 1), connecting the new points
to the original mesh (step 1), flipping the original edges to obtain a new
set of faces (step 3). Step 2 involves shifting the original vertices and is
not shown.

For this task, you will implement the subdivision scheme described in
[Kobbelt00sqrt(3)-subdivision](https://www.graphics.rwth-aachen.de/media/papers/sqrt31.pdf) to
iteratively create finer meshes from a given coarse mesh.
According to the paper, given a mesh `(V, F)`, the √3-subdivision scheme creates a new mesh `(V', F')`
by applying the following rules:

 1. Add a new vertex at location `m_f` for each face `f` in `F` of the original mesh.
    The new vertex will be located at the midpoint of the face.
    Append the newly created vertices `M = {m_f}` to `V` to create a new set of vertices `V'' = [V; M]`.
    Add three new faces for each face f in order by connecting `m_f` with edges to the original 3
    vertices of the face; we call the set of this newly created faces `F''`.
    <!-- Replace the old set of faces `F` with `F''`. -->
 2. Move each vertex `v` of the old vertices `V` to a new position `p` by averaging `v` with the positions of its neighboring vertices in the *original* mesh.
    If `v` has valence `n` and its neighbors in the original mesh `(V, F)` are `v_0`, `v_1`, ...,  `v_n`, then the update rule is<br/>
    ![](https://latex.codecogs.com/png.latex?\fg_gray%20p=(1-a_n)v&plus;\frac{a_n}{n}\sum\limits_{i=0}^{n-1}v_i)<br/>
    <!-- p = (1-a_n) v + \frac{a_n}{n}\sum\limits_{i=0}^{n-1} v_i -->where<br/>
    ![](https://latex.codecogs.com/png.latex?\fg_gray%20a_n=\frac{4-2\cos\left(\frac{2\pi}{n}\right)}{9}.)<br/>
    <!-- a_n=\frac{4-2\cos\left(\frac{2\pi}{n}\right)}{9} -->
    The vertex set of the subdivided mesh is then `V' = [P, M]`, where `P` is the concatenation of the new positions `p` for all vertices.
  3. Replace the `F''` with a new set of faces `F'` such that the edges connecting the newly added points `M` to `P`
    (the relocated original vertices) remain but the original edges of the mesh connecting points in `M` to each other are flipped.



![](img/subdivision.png?raw=true)

Fill in the appropriate source code sections so that hitting the mesh is subdivided and displayed.

*Relevant `igl` functions:* Many options possible.
Some suggestions: `adjacency_list`, `triangle_triangle_adjacency`, `edge_topology`, `barycenter`.


Required output of this section:

* Screenshots of the subdivided meshes.